In [48]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [49]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

In [50]:
# input_shape = Input(shape=(32,32,3))
# # convlution layers
# X = Conv2D(32, (3, 3), activation='relu', padding='same')(input_shape)
# X = Conv2D(32, (3, 3), padding='same')(X)
# # shortcut
# shortcut = Conv2D(32, (1, 1), padding='same')(input_shape)
# X = Add()([X,shortcut])
# X= ReLU()(X)
# model = Model(inputs=input_shape, outputs=X)
# model.summary()



In [51]:
from tensorflow.keras.layers import Input, Conv2D, Add, ReLU
from tensorflow.keras.models import Model

# Define input
input_tensor = Input(shape=(32, 32, 3))

# Convolutional layers (same output shape as input)
x = Conv2D(64, (3,3), padding="same", activation="relu")(input_tensor)
x = Conv2D(64, (3,3), padding="same")(x)

# shortcut
shortcut = Conv2D(64, (1,1), padding="same")(input_tensor)
x = Add()([x, shortcut])
x = ReLU()(x)  # Activation after addition
x = Flatten()(x)
x = Dense(10, activation="softmax")(x)

# Create model
model = Model(inputs=input_tensor, outputs=x)
model.summary()


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25            │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_149 (Conv2D)       │ (None, 32, 32, 64)     │          1,792 │ input_layer_25[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_150 (Conv2D)       │ (None, 32, 32, 64)     │         36,928 │ conv2d_149[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_151 (Conv2D)       │ (None, 32, 32, 64)     │            256 │ input_layer_25[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_59 (Add)              │ (None, 32, 32, 64)     │              0 │ conv2d_150[0][0],      │
│                           │                        │                │ conv2d_151[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_92 (ReLU)           │ (None, 32, 32, 64)     │              0 │ add_59[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 65536)          │              0 │ re_lu_92[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 10)             │        655,370 │ flatten_2[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 694,346 (2.65 MB)

 Trainable params: 694,346 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [53]:
model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_test,y_test))

Epoch 1/10
1037/1563 ━━━━━━━━━━━━━━━━━━━━ 1:48 207ms/step - accuracy: 0.3892 - loss: 1.7332

KeyboardInterrupt: 

In [34]:
def residual_block(x,filters,downsample = False):
  shortcut = x
  stride = 2 if downsample else 1
  # layer1
  x = Conv2D(filters,(3,3),strides=stride,padding='same')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  # layer2
  x = Conv2D(filters,(3,3),strides=1,padding='same')(x)
  x = BatchNormalization()(x)
  # condition
  if downsample:
    shortcut = Conv2D(filters,(1,1),strides=2,padding='same')(shortcut)
  x = Add()([x,shortcut])
  x = ReLU()(x)
  return x


In [45]:
def Resnet18(num_classes=10,input_shape=(32,32,3)):
  inputs = Input(shape=input_shape)
  x = Conv2D(64,(7,7),strides=2,padding='same')(inputs)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(x)
  # blocks1
  x = residual_block(x,64)
  x = residual_block(x,64)
  # blocks2
  x = residual_block(x,128,downsample=True)
  x = residual_block(x,128)
  # blocks3
  x = residual_block(x,256,downsample=True)
  x = residual_block(x,256)
  # blocks4
  x = residual_block(x,512,downsample=True)
  x = residual_block(x,512)
  x=GlobalAveragePooling2D()(x)
  outputs = Dense(num_classes,activation='softmax')(x)
  model = Model(inputs,outputs,name="ResNet18")
  return model

In [46]:
model = Resnet18()
model.summary()
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "ResNet18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24            │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_129 (Conv2D)       │ (None, 16, 16, 64)     │          9,472 │ input_layer_24[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_68    │ (None, 16, 16, 64)     │            256 │ conv2d_129[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_75 (ReLU)           │ (None, 16, 16, 64)     │              0 │ batch_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 8, 8, 64)       │              0 │ re_lu_75[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_130 (Conv2D)       │ (None, 8, 8, 64)       │         36,928 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_69    │ (None, 8, 8, 64)       │            256 │ conv2d_130[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_76 (ReLU)           │ (None, 8, 8, 64)       │              0 │ batch_normalization_6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_131 (Conv2D)       │ (None, 8, 8, 64)       │         36,928 │ re_lu_76[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_70    │ (None, 8, 8, 64)       │            256 │ conv2d_131[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_51 (Add)              │ (None, 8, 8, 64)       │              0 │ batch_normalization_7… │
│                           │                        │                │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_77 (ReLU)           │ (None, 8, 8, 64)       │              0 │ add_51[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_132 (Conv2D)       │ (None, 8, 8, 64)       │         36,928 │ re_lu_77[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_71    │ (None, 8, 8, 64)       │            256 │ conv2d_132[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_78 (ReLU)           │ (None, 8, 8, 64)       │              0 │ batch_normalization_7… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_133 (Conv2D)       │ (None, 8, 8, 64)       │         36,928 │ re_lu_78[0][0]         │
├──────────────────────

 Total params: 11,192,458 (42.70 MB)

 Trainable params: 11,184,650 (42.67 MB)

 Non-trainable params: 7,808 (30.50 KB)

In [47]:
model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_test,y_test))

Epoch 1/10
 321/1563 ━━━━━━━━━━━━━━━━━━━━ 19:06 923ms/step - accuracy: 0.2838 - loss: 2.2888

KeyboardInterrupt: 